In [1]:
# Cell 1: Install Dependencies
!pip install -q transformers accelerate pillow pandas torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.7 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 26.9 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed.

In [2]:
# Cell 2: Import Libraries
import os
import pandas as pd
from PIL import Image
from transformers import AutoTokenizer, AutoProcessor, AutoModelForImageTextToText
from pathlib import Path
import torch
from tqdm.auto import tqdm
import warnings
import gc
warnings.filterwarnings('ignore')

2025-11-30 19:28:52.778769: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764530933.178649      91 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764530933.297308      91 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
# Cell 3: Load Nanonets OCR Model (Optimized)
print("Loading Nanonets OCR Model with optimizations...")

model_path = "nanonets/Nanonets-OCR-s"

# Set device explicitly to cuda:0 to avoid multi-GPU issues
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load model on single GPU
model = AutoModelForImageTextToText.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    low_cpu_mem_usage=True
).to(device)

model.eval()

# Enable optimizations
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    torch.backends.cuda.matmul.allow_tf32 = True

tokenizer = AutoTokenizer.from_pretrained(model_path)
processor = AutoProcessor.from_pretrained(model_path)

print("Model loaded successfully!")
print(f"Model device: {device}")
print(f"Mixed precision: FP16 enabled")

Loading Nanonets OCR Model with optimizations...
Using device: cuda:0


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.51G [00:00<?, ?B/s]

...

chat_template.jinja: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


video_preprocessor_config.json: 0.00B [00:00, ?B/s]

Model loaded successfully!
Model device: cuda:0
Mixed precision: FP16 enabled


In [4]:
# Cell 4: Define Optimized OCR Function
def extract_text_from_image(image_path, model, processor, max_new_tokens=1024):
    """
    Optimized OCR extraction with reduced token limit for speed
    """
    try:
        # Open and resize large images for faster processing
        image = Image.open(image_path).convert('RGB')
        
        # Resize if too large (speeds up processing significantly)
        max_size = 1280
        if max(image.size) > max_size:
            ratio = max_size / max(image.size)
            new_size = tuple(int(dim * ratio) for dim in image.size)
            image = image.resize(new_size, Image.LANCZOS)
        
        # Optimized prompt (shorter for faster processing)
        prompt = """Extract all text from this image including Bengali, English, and Banglish text. Return only the extracted text."""

        messages = [
            {"role": "system", "content": "You are an OCR assistant."},
            {
                "role": "user", 
                "content": [
                    {"type": "image", "image": str(image_path)},
                    {"type": "text", "text": prompt}
                ]
            }
        ]
        
        text = processor.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=True
        )
        
        inputs = processor(
            text=[text], 
            images=[image], 
            padding=True, 
            return_tensors="pt"
        )
        
        # Move all inputs to the same device as model (cuda:0)
        device = next(model.parameters()).device
        inputs = {k: v.to(device) if hasattr(v, 'to') else v for k, v in inputs.items()}
        
        # Fast generation with optimizations
        with torch.no_grad(), torch.amp.autocast('cuda'):
            output_ids = model.generate(
                **inputs, 
                max_new_tokens=max_new_tokens,
                do_sample=False,
                num_beams=1,
                pad_token_id=processor.tokenizer.pad_token_id,
                use_cache=True
            )
        
        generated_ids = [
            output_ids[len(input_ids):] 
            for input_ids, output_ids in zip(inputs['input_ids'], output_ids)
        ]
        
        output_text = processor.batch_decode(
            generated_ids, 
            skip_special_tokens=True, 
            clean_up_tokenization_spaces=True
        )
        
        extracted_text = output_text[0].strip() if output_text else ""
        
        # Clean up
        if "Extract all text" in extracted_text:
            extracted_text = extracted_text.split("Return only the extracted text.")[-1].strip()
        
        # Clear cache
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        return extracted_text
        
    except Exception as e:
        print(f"Error: {image_path.name}: {str(e)}")
        return ""

In [5]:
# Cell 5: Checkpoint Saving Function
def save_checkpoint(results, checkpoint_num, output_dir="/kaggle/working"):
    """Save checkpoint CSV"""
    checkpoint_path = f"{output_dir}/checkpoint_{checkpoint_num:04d}.csv"
    df_checkpoint = pd.DataFrame(results)
    df_checkpoint.to_csv(checkpoint_path, index=False, encoding='utf-8-sig')
    print(f"💾 Checkpoint saved: {checkpoint_path} ({len(results)} images)")
    return checkpoint_path

def load_latest_checkpoint(output_dir="/kaggle/working"):
    """Load latest checkpoint if exists"""
    checkpoints = list(Path(output_dir).glob("checkpoint_*.csv"))
    if checkpoints:
        latest = max(checkpoints, key=lambda x: x.stat().st_mtime)
        df = pd.read_csv(latest, encoding='utf-8-sig')
        print(f"📂 Loaded checkpoint: {latest.name} ({len(df)} images)")
        return df.to_dict('records'), len(df)
    return [], 0

In [6]:
# Cell 6: Process All Images with Checkpointing (OPTIMIZED)
import time

# Define paths
image_dir = "/kaggle/input/poli-meme-decode-cuet-cse-fest/PoliMemeDecode/Test/Image"
output_dir = "/kaggle/working"
checkpoint_interval = 5  # Save every 5 images

# Get all image files
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif', '.webp']
image_files = []

for ext in image_extensions:
    image_files.extend(list(Path(image_dir).glob(f"*{ext}")))
    image_files.extend(list(Path(image_dir).glob(f"*{ext.upper()}")))

image_files = sorted(image_files)

print(f"📁 Found {len(image_files)} images to process")

# Try to resume from checkpoint
results, start_idx = load_latest_checkpoint(output_dir)
processed_files = {r['image_filename'] for r in results}

print(f"▶️  Starting from image {start_idx + 1}/{len(image_files)}\n")

# Track performance
start_time = time.time()
checkpoint_counter = 0

# Process images
with tqdm(total=len(image_files), initial=start_idx, desc="OCR Progress") as pbar:
    for idx, img_path in enumerate(image_files):
        filename = img_path.name
        
        # Skip if already processed
        if filename in processed_files:
            pbar.update(1)
            continue
        
        # Extract text
        extracted_text = extract_text_from_image(img_path, model, processor)
        
        results.append({
            'image_filename': filename,
            'extracted_text': extracted_text,
            'text_length': len(extracted_text),
            'has_text': len(extracted_text) > 0
        })
        
        checkpoint_counter += 1
        
        # Save checkpoint every 5 images
        if checkpoint_counter % checkpoint_interval == 0:
            save_checkpoint(results, len(results), output_dir)
            
            # Calculate speed
            elapsed = time.time() - start_time
            speed = len(results) / elapsed
            eta = (len(image_files) - len(results)) / speed if speed > 0 else 0
            
            pbar.set_postfix({
                'speed': f'{speed:.1f} img/s',
                'ETA': f'{eta/60:.1f}min'
            })
        
        pbar.update(1)
        
        # Memory cleanup every 20 images
        if checkpoint_counter % 20 == 0:
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

# Final checkpoint
if checkpoint_counter % checkpoint_interval != 0:
    save_checkpoint(results, len(results), output_dir)

elapsed_time = time.time() - start_time
print(f"\n✅ Completed {len(results)} images in {elapsed_time/60:.2f} minutes")
print(f"⚡ Average speed: {len(results)/elapsed_time:.2f} images/second")

📁 Found 330 images to process
▶️  Starting from image 1/330



OCR Progress:   0%|          | 0/330 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


💾 Checkpoint saved: /kaggle/working/checkpoint_0005.csv (5 images)


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...

💾 Checkpoint saved: /kaggle/working/checkpoint_0320.csv (320 images)


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


💾 Checkpoint saved: /kaggle/working/checkpoint_0325.csv (325 images)


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


💾 Checkpoint saved: /kaggle/working/checkpoint_0330.csv (330 images)

✅ Completed 330 images in 83.22 minutes
⚡ Average speed: 0.07 images/second


In [7]:
# Cell 7: Create Final DataFrame and Save
print("\n📊 Creating final CSV files...")

# Create DataFrame
df = pd.DataFrame(results)

# Save main output
output_path = f"{output_dir}/extracted_texts_final.csv"
df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\n✅ Final CSV saved: {output_path}")
print(f"\n📊 Dataset Statistics:")
print(f"━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
print(f"  Total images: {len(df)}")
print(f"  Images with text: {df['has_text'].sum()}")
print(f"  Images without text: {(~df['has_text']).sum()}")
print(f"  Success rate: {df['has_text'].sum()/len(df)*100:.1f}%")
print(f"  Average text length: {df['text_length'].mean():.2f} characters")
print(f"  Max text length: {df['text_length'].max()} characters")
print(f"━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")


📊 Creating final CSV files...

✅ Final CSV saved: /kaggle/working/extracted_texts_final.csv

📊 Dataset Statistics:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  Total images: 330
  Images with text: 330
  Images without text: 0
  Success rate: 100.0%
  Average text length: 286.05 characters
  Max text length: 2252 characters
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


In [8]:
# Cell 8: Language Analysis and Enhanced Output
def detect_language_mix(text):
    """Detect language composition"""
    has_bangla = any('\u0980' <= char <= '\u09FF' for char in text)
    has_english = any(char.isalpha() and char.isascii() for char in text)
    has_digits = any(char.isdigit() for char in text)
    return has_bangla, has_english, has_digits

print("\n🔍 Analyzing language distribution...")

df['has_bangla'] = df['extracted_text'].apply(lambda x: detect_language_mix(x)[0])
df['has_english'] = df['extracted_text'].apply(lambda x: detect_language_mix(x)[1])
df['has_digits'] = df['extracted_text'].apply(lambda x: detect_language_mix(x)[2])
df['is_mixed'] = df['has_bangla'] & df['has_english']

print(f"\n🌐 Language Distribution:")
print(f"━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
print(f"  Bangla: {df['has_bangla'].sum()} ({df['has_bangla'].sum()/len(df)*100:.1f}%)")
print(f"  English: {df['has_english'].sum()} ({df['has_english'].sum()/len(df)*100:.1f}%)")
print(f"  Mixed: {df['is_mixed'].sum()} ({df['is_mixed'].sum()/len(df)*100:.1f}%)")
print(f"  With digits: {df['has_digits'].sum()} ({df['has_digits'].sum()/len(df)*100:.1f}%)")
print(f"━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")

# Save enhanced version
output_path_enhanced = f"{output_dir}/extracted_texts_enhanced.csv"
df.to_csv(output_path_enhanced, index=False, encoding='utf-8-sig')
print(f"\n✅ Enhanced CSV saved: {output_path_enhanced}")


🔍 Analyzing language distribution...

🌐 Language Distribution:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  Bangla: 177 (53.6%)
  English: 221 (67.0%)
  Mixed: 126 (38.2%)
  With digits: 65 (19.7%)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

✅ Enhanced CSV saved: /kaggle/working/extracted_texts_enhanced.csv


In [9]:
# Cell 9: Sample Output Display
print("\n" + "="*80)
print("📝 SAMPLE EXTRACTED TEXTS")
print("="*80 + "\n")

# Show diverse samples (with text, without text, Bangla, English, Mixed)
samples = []
if any(df['has_bangla'] & ~df['has_english']):
    samples.append(df[df['has_bangla'] & ~df['has_english']].iloc[0])
if any(df['has_english'] & ~df['has_bangla']):
    samples.append(df[df['has_english'] & ~df['has_bangla']].iloc[0])
if any(df['is_mixed']):
    samples.append(df[df['is_mixed']].iloc[0])
if any(~df['has_text']):
    samples.append(df[~df['has_text']].iloc[0])

# Add first few
for row in df.head(3).to_dict('records'):
    if row not in samples:
        samples.append(row)

for row in samples[:8]:
    if isinstance(row, pd.Series):
        row = row.to_dict()
    
    print(f"📷 {row['image_filename']}")
    print(f"   Stats: {row['text_length']} chars | " +
          f"Bangla: {'✓' if row.get('has_bangla', False) else '✗'} | " +
          f"English: {'✓' if row.get('has_english', False) else '✗'} | " +
          f"Mixed: {'✓' if row.get('is_mixed', False) else '✗'}")
    
    if row['extracted_text']:
        text = row['extracted_text'][:150]
        if len(row['extracted_text']) > 150:
            text += "..."
        print(f"   Text: {text}")
    else:
        print(f"   Text: [No text detected]")
    print("-" * 80)

print("\n✅ All done! Output files:")
print(f"   • {output_path}")
print(f"   • {output_path_enhanced}")
print(f"   • Multiple checkpoints saved every 5 images")


📝 SAMPLE EXTRACTED TEXTS



ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
# Cell 10: Cleanup Checkpoints (Optional)
cleanup = input("\n🧹 Delete checkpoint files? (y/n): ").lower()
if cleanup == 'y':
    checkpoints = list(Path(output_dir).glob("checkpoint_*.csv"))
    for cp in checkpoints:
        cp.unlink()
    print(f"✅ Deleted {len(checkpoints)} checkpoint files")
else:
    print("📁 Checkpoint files preserved")